In [1]:
import pandas as pd
import numpy as np
import json
import pyarrow
import logging
import os
from collections import Counter
import nbimporter
from Profile_based_recommender import User_profile_recommender

Importing Jupyter notebook from Profile_based_recommender.ipynb


In [5]:
#Spark imports and setting spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark

spark = SparkSession.builder.master('local[*]').appName('Recommender_Engine1').getOrCreate()

Exception: Java gateway process exited before sending its port number

In [ ]:
#Setting the configurations like paths of the files
config_file = open("config.json")
configs  = json.load(config_file)

#Setting the logging files
logging.basicConfig(filename = os.path.join(configs['local_paths']['logging_path'] , 'log.txt') , filemode = 'w' , level = 'INFO')

In [ ]:
analytics_sandbox_path = configs['local_paths']['analytics_sandbox']
df = spark.read.csv(analytics_sandbox_path, header = True, inferSchema = True)
user_profile = df.filter(df.user_id == 'rGd8YUtvhSdMm8-9cMdQ4Q')
df = df.filter(df.city == 'Montreal')
temp_df  = df.toPandas()

In [ ]:
print(spark)

In [ ]:
pandas_df = temp_df.iloc[ : 5000 ,  : ]
test_df = temp_df.iloc[ 5001 :  , : ]

In [ ]:
restaurant_df = pandas_df[(pandas_df.business_categories.str.contains('Restaurants') ) 
                          & (pandas_df.user_review_count > 10)  
                          & (pandas_df.review_count > 10)]
matrix_x = pd.pivot_table(data=restaurant_df , index='user_id', columns='business_id' , values='review_stars')

In [ ]:
user_profile = user_profile.toPandas()

In [ ]:
if __name__ == "__main__":
    '''set_spark_session()
    set_configs()'''
    ur = User_profile_recommender(user_profile , restaurant_df )
    user_recommendations = ur.get_recommendations()